<a href="https://colab.research.google.com/github/carolinehagood/ds4002-project3/blob/main/project3_loadingdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image

In [ ]:

from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import zipfile
import os



Mounted at /content/drive


In [ ]:
# Path to the zip file in Google Drive
zip_path = '/content/drive/MyDrive/dogimages.zip'
extract_path = '/content/Dogs'  # Choose a location in Colab to extract the files

# Unzip the file
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Files unzipped successfully.")
except FileNotFoundError:
    print("Zip file not found at the specified path.")
except zipfile.BadZipFile:
    print("The file is not a zip file or is corrupted.")
except Exception as e:
    print(f"An error occurred: {e}")


Files unzipped successfully.


In [ ]:
images_dir = '/content/Dogs/dog-images'


In [ ]:
#function to load images and create dataframe
import re
from sklearn.preprocessing import LabelEncoder

def load_images_with_labels(images_dir, img_size=(128, 128)):
    image_data = []
    breed_labels = []

    # Get list of breed folders
    breed_names = sorted(os.listdir(images_dir))

    for breed in breed_names:
        breed_folder_path = os.path.join(images_dir, breed)

        # Only proceed if it's a directory
        if not os.path.isdir(breed_folder_path):
            continue

        for image_file in os.listdir(breed_folder_path):
            image_path = os.path.join(breed_folder_path, image_file)

            # Check if the current file is an image
            if os.path.isfile(image_path) and image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    # Open and resize the image
                    img = Image.open(image_path).resize(img_size)
                    img = np.array(img)

                    # Check if image has three color channels (RGB)
                    if img.shape == (img_size[0], img_size[1], 3):
                        # Flatten the image (convert 128x128x3 to 1D array)
                        image_data.append(img.flatten())
                        breed_labels.append(breed)
                except Exception as e:
                    print(f"Could not load image {image_path}: {e}")

    # Convert lists to numpy arrays for efficient processing
    image_data = np.array(image_data)

    # Normalize image data (scaling pixel values between 0 and 1)
    image_data = image_data / 255.0

    # Apply LabelEncoder to convert breed names to numeric labels
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(breed_labels)

    # Create a DataFrame for organized data
    data_df = pd.DataFrame(image_data)

    # Add the encoded labels as a new column
    data_df['breed'] = encoded_labels

    # Add breed names as a new column (optional)
    data_df['breed_name'] = data_df['breed'].apply(lambda x: label_encoder.inverse_transform([x])[0])

    return data_df, image_data, encoded_labels, label_encoder


# Path to extracted images directory
images_dir = '/content/Dogs/dog-images'

# Load images and labels
data_df, image_data, encoded_labels, label_encoder = load_images_with_labels(images_dir)


In [ ]:
print(data_df.head(2))


          0         1         2         3         4         5         6  \
0  0.074510  0.070588  0.054902  0.066667  0.062745  0.047059  0.070588   
1  0.043137  0.066667  0.058824  0.047059  0.070588  0.062745  0.047059   

          7         8         9  ...     49144     49145     49146     49147  \
0  0.066667  0.050980  0.125490  ...  0.325490  0.301961  0.368627  0.349020   
1  0.070588  0.062745  0.058824  ...  0.011765  0.015686  0.380392  0.043137   

      49148     49149     49150     49151  breed             breed_name  
0  0.321569  0.380392  0.364706  0.321569      0  n02085936-Maltese_dog  
1  0.054902  0.250980  0.023529  0.031373      0  n02085936-Maltese_dog  

[2 rows x 49154 columns]


In [ ]:
#testing data set

print(f"Image data shape: {image_data.shape}")


Image data shape: (541, 49152)


In [ ]:
#number of rows of dataframe
len(data_df)

541

In [ ]:
# Save the DataFrame to an HDF5 file
data_df.to_hdf('/content/data_df.h5', key='df', mode='w')

# Download the HDF5 file
from google.colab import files
files.download('/content/data_df.h5')


<ipython-input-36-60c3670a403b>:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  data_df.to_hdf('/content/data_df.h5', key='df', mode='w')
<ipython-input-36-60c3670a403b>:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_items] [items->None]

  data_df.to_hdf('/content/data_df.h5', key='df', mode='w')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>